In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append(".")
sys.path.append("..")
from quantile_binary_search.method import quantile_binary_search
from kaplan_et_al.single_quantile_algo import kaplan_quantile
from bisect import bisect_right

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Rank error comparison for median selection

In [2]:
labels = ["algo", "input", "d", "n", "rho", "mean", "sigma", "rel", "abs"]

df = pd.DataFrame(columns=labels)

In [3]:
ds = [10, 100, 1000, 4000]
n = 4000
mean = 10
sigma = 1
rho = .1
trials = 100

algos = {
    "kaplan": lambda data, n, lower, upper, rho: kaplan_quantile(data, (lower, upper), .5, rho, None),
    "binary_search": lambda data, n, lower, upper, rho: quantile_binary_search(data, .5*n, upper, rho, l=lower),
}

inputs = {
    "normal": lambda: np.sort(np.random.normal(mean, sigma, size=n)),
    "steps": lambda: [mean - 5 * sigma + i * 10 * sigma / n for i in range(n)],
}

def compare_rank_error(data, rho, lower, upper, f):
    n = len(data)
    x = f(data, n, lower, upper, rho)
    return bisect_right(data, x)


#data = np.sort(np.random.normal(mean, sigma, size=n))
j = 0
for input in inputs:
    data = inputs[input]()
    for d in ds:
        for algo, f in algos.items():
            for i in range(trials):
                idx = compare_rank_error(data, rho / (8 * d), mean - 5 * sigma, mean + 5 * sigma, f)
                df = pd.concat([df, pd.DataFrame({
                    "algo": algo,
                    "input": input,
                    "d": d,
                    "n": n,
                    "rho": rho / (8 * d),
                    "mean": mean,
                    "sigma": sigma,
                    "rel": idx / n,
                    "abs": np.abs(.5 * n - idx)
                }, index=[j])])
                j += 1


In [4]:
df[["algo", "input", "rho", "rel", "abs", "sigma"]].groupby(["input", "rho", "sigma", "algo"]).mean()

rel     abs
input  rho      sigma algo                           
normal 0.000003 1     binary_search  0.520478  736.37
                      kaplan         0.498062  228.53
       0.000013 1     binary_search  0.472242  315.61
                      kaplan         0.504085  105.60
       0.000125 1     binary_search  0.502518  105.57
                      kaplan         0.498308   33.95
       0.001250 1     binary_search  0.499990   38.04
                      kaplan         0.499868   10.65
steps  0.000003 1     binary_search  0.495175  469.60
                      kaplan         0.507373  218.73
       0.000013 1     binary_search  0.480440  310.32
                      kaplan         0.502685  101.38
       0.000125 1     binary_search  0.497940  105.60
                      kaplan         0.500498   32.43
       0.001250 1     binary_search  0.499522   29.11
                      kaplan         0.500400    9.56

In [5]:
import math

d = 4000

b = mean + 5 * sigma
a = mean - 5 * sigma

print("n", n, "sigma", sigma, "rho", rho, "d", d, "adapted rho", rho / (8 * d))

min_diff_between_pairs = 1/sigma * 1/n**2 

psi = (b - a) / min_diff_between_pairs

rank_error = 1/math.sqrt(rho / (8 * d)) * math.log(psi)

rank_error

n 4000 sigma 1 rho 0.1 d 4000 adapted rho 3.125e-06


10686.184817234498

In [6]:
# compare rank errors

def binary_search_rank_error(d, n, R, sigma_min, rho, beta=0.1):
    from math import sqrt, log
    u = 2 * R * sqrt(n) / sigma_min
    tau = sqrt(log(d * u) * log(4 * d * log(d * u)/ beta)/rho)
    return tau

def quantile_rank_error(d, n, R, sigma_min, rho, beta=0.1):
    from math import sqrt, log
    psi = R * n**2 * 1/sigma_min
    return (log(psi) + log (1/beta)) / sqrt(rho)
    
u = mean + 5 * sigma
rho_per = rho / (8 * d)
beta = 0.5 / d

sigma_min = sigma

print(d, n, rho, beta, "binary search", 
    binary_search_rank_error(d, n, u, sigma_min, rho_per, beta),
    "kaplan", quantile_rank_error(d, n, u, sigma_min, rho_per, beta))

4000 4000 0.1 0.000125 binary search 10423.103759073287 kaplan 15999.476771815362


# Compare standard deviations

In [9]:
ds = [128]
n = 4000
mean = 10
rhos = [0.1, 1.]
trials = 100

df = pd.DataFrame()


algos = {
    "direct-075": lambda data, n, lower, upper, rho: direct_estimation(data, rho, lower, upper, 0.75),
    "direct-0841": lambda data, n, lower, upper, rho: direct_estimation(data, rho, lower, upper, 0.841),
    "direct-09": lambda data, n, lower, upper, rho: direct_estimation(data, rho, lower, upper, 0.9),
    "robust-kgroups1": lambda data, n, lower, upper, rho: robust_estimation(data, rho, lower, upper),
    "robust-kgroups4": lambda data, n, lower, upper, rho: robust_estimation(data, rho, lower, upper, k_groups=4),
    "robust-kgroups16": lambda data, n, lower, upper, rho: robust_estimation(data, rho, lower, upper, k_groups=16),
}

inputs = {
    "normal": lambda: np.sort(np.random.normal(mean, sigma, size=n)),
}

def direct_estimation(data, rho, lower, upper, q):
    n = len(data)
    mean = kaplan_quantile(data, (lower, upper), .5, rho/2, False)
    x = kaplan_quantile(data, (lower, upper), q, rho/2, False)
    return x - mean

def robust_estimation(data, rho, lower, upper, k_groups=1):
    upper *= upper
    rho /= 2 # half of the rho budget is used for mean estimation
    rng = np.random.default_rng()
    rng.shuffle(data)
    odd, even = data[::2], data[1::2]
    pairwise = np.array([0.5* (x-y)**2 for (x, y) in zip(odd, even)])
    # Group into list of tuples to sum
    groups = list(zip(*[iter(pairwise)] * k_groups))
    # Sum and divide by k
    robust_std_estimates = np.array(list(map(lambda s: s/k_groups, map(sum, groups))))
    if True:
        for _ in range(2 * k_groups - 1):
            rng = np.random.default_rng()
            rng.shuffle(data)
            odd, even = data[::2], data[1::2]
            pairwise = np.array([0.5* (x-y)**2 for (x, y) in zip(odd, even)]) 
            # Group into list of tuples to sum
            groups = list(zip(*[iter(pairwise)]*k_groups))
            # Sum and divide by k
            robust_std_estimates = np.concatenate((robust_std_estimates, 
                np.array(list(map(lambda s: s/k_groups, map(sum, groups))))))
        rho /= 2 * k_groups
    std_predictions = np.sqrt(kaplan_quantile(robust_std_estimates, 
        (0, upper), .5, rho, False) / (1-2/(9*k_groups))**3 )
    return std_predictions



#data = np.sort(np.random.normal(mean, sigma, size=n))
j = 0
for sigma in [0.001, 1]:
    for rho in rhos:
        for input in inputs:
            data = inputs[input]()
            for d in ds:
                for algo, f in algos.items():
                    for i in range(trials):
                        estimate = f(data, n,  mean - 5 * sigma, mean + 5 * sigma, rho / (8 * d))
                        df = pd.concat([df, pd.DataFrame({
                            "algo": algo,
                            "input": input,
                            "d": d,
                            "n": n,
                            "rho": rho / (8 * d),
                            "mean": mean,
                            "sigma": sigma,
                            "rel-err": np.abs(sigma - estimate)/sigma
                        }, index=[j])])
                        j += 1

In [10]:
df[["algo", "input", "rho", "rel-err", "sigma"]].groupby(["input", "rho", "sigma", "algo"]).mean()

rel-err
input  rho      sigma algo                         
normal 0.000098 0.001 direct-075           0.344891
                      direct-0841          0.186755
                      direct-09            0.704214
                      robust-kgroups1      0.043536
                      robust-kgroups16  6569.225194
                      robust-kgroups4   6761.211072
                1.000 direct-075           0.314580
                      direct-0841          0.162470
                      direct-09            0.585862
                      robust-kgroups1      0.045050
                      robust-kgroups16     5.715187
                      robust-kgroups4      0.027169
       0.000977 0.001 direct-075           0.333706
                      direct-0841          0.044746
                      direct-09            0.245398
                      robust-kgroups1      0.024843
                      robust-kgroups16  3009.511496
                      robust-kgroups4      0.013180
                1.000 direct-075           0.334036
                      direct-0841          0.030342
                      direct-09            0.284093
                      robust-kgroups1      0.025993
                      robust-kgroups16     0.011265
                      robust-kgroups4      0.012270